In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-22 09:42:54,473 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at_raw = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
# HOTFIX - remove odd day
at_raw = at_raw[~at_raw['DAY'].isin([9])]

In [5]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2447675424832) was called 1 times


In [6]:
at = feat.pre_process_AT(at_raw)

Running: Function "pre_process_AT" (id=2447675427712) was called 1 times


In [7]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2447675428576) was called 1 times


In [8]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2447675427280) was called 1 times
Running: Function "load_module_lookup" (id=2447675426128) was called 1 times
Running: Function "load_ID_lookup" (id=2447675426848) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [9]:
fa.loc[fa['Alert'].str.contains(r'C[0-9]{4}PTT[0-9]{3}'), 'Asset Code'] = fa.loc[fa['Alert'].str.contains(r'C[0-9]{4}PTT[0-9]{3}')]['Alert'].str.extract('(C[0-9]{4}PTT[0-9]{3})')[0].str.replace('02', '')

In [10]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [11]:
fa['MODULE']

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
154357,107560.0,C13: C1302RDC086 086AQ1 roller motor unit fault,2020-11-22 06:29:00,C13,P02,1.0,3429,C1302RDC086,Grey,Module,C1302,False,13.0,4.0,17,Fault
154358,50586.0,C06: C0602RDC036 036AQ2 roller motor unit fault,2020-11-22 05:39:00,C06,P02,1.0,3429,C0602RDC036,Blue,Module,C0602,False,6.0,1.0,2,Fault
159975,430945.0,SCS183: main switch is OFF in automaticmode X:...,2020-11-25 01:49:00,SCSM18,SCS183,1.0,5032,SCS183,Grey,carousel,SCSM18,False,0.0,4.0,18,Fault
154360,50586.0,C06: C0602RDC036 036AQ2 roller motor unit fault,2020-11-22 05:38:00,C06,P02,1.0,3429,C0602RDC036,Blue,Module,C0602,False,6.0,1.0,2,Fault
159972,430721.0,SCS182: main switch is OFF in automaticmode X:...,2020-11-25 01:49:00,SCSM18,SCS182,1.0,5032,SCS182,Grey,carousel,SCSM18,False,0.0,4.0,18,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19295,141205.0,C17: C1701RDC251 251B11 occupancy sensor fault,2020-11-14 14:48:00,C17,P01,64249.0,3416,C1701RDC251,Blue,NaN,NaN,False,17.0,0.0,NaN,Fault
131073,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-11-24 05:50:00,C16,P02,65301.0,3430,C1602ACL040,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault
81842,188345.0,C23: C2303BCL151 151B54 sensor fault,2020-11-22 05:26:00,C23,P03,65901.0,3428,C2303BCL151,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
20149,141205.0,C17: C1701RDC251 251B11 occupancy sensor fault,2020-11-13 10:27:00,C17,P01,72401.0,3416,C1701RDC251,Blue,NaN,NaN,False,17.0,0.0,NaN,Fault


In [ ]:
mod = [7]
limit = 250

In [ ]:
fa_sel_1 = feat.get_data_faults(fa_floor, modules = mod)

most_common = list(((fa_sel_1['Asset Code'].value_counts() > limit)[fa_sel_1['Asset Code'].value_counts() > limit]).index)
fa_sel = fa_sel_1[fa_sel_1['Asset Code'].isin(most_common)]

In [ ]:
#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'Asset Code', agg_type = 'count')

In [ ]:
av_sel, at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = {"Module" : mod})

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

In [ ]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, agg_level = 'None')

In [ ]:
# df = (pd.merge(at_agg.reset_index(), av_agg.reset_index(), on=['timestamp', 'Module'], how='left')
#  .merge(fa_agg, on=['timestamp'], how='left')).drop(['Module', 'Blue Tote Loss', 'Grey Tote Loss'], axis=1)
# df = df.fillna(0)

In [ ]:
X,y = md.gen_feat_var(df)
X = X.drop('Module', axis=1)
# Flip to availability
y=1-y

In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

#XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test)

maxf = None

# X_sel_rf = md.select_features(X, y, RF_mdl, thres = -np.inf, max_feat = maxf)

#X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)


cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 5, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(RF_mdl, X_sel_rf, y, n_folds = 10, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(XGB_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

#md.stats_model(Linear_mdl,X_test,y_test)

In [ ]:
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

In [ ]:
keep_features = results.pvalues[results.pvalues < 0.10].index

In [ ]:
model = sm.OLS(y,X[keep_features])
results = model.fit()
print(results.summary())

In [ ]:
negs = results.params[results.params < 0].index
negs

In [ ]:
df['timestamp'].min()

In [ ]:
fa[fa['Asset Code'].isin([negs]) & (fa['timestamp'] >= df['timestamp'].min())]#['Fault ID'].value_counts()